In [39]:
# 파이썬 라이브러리 중 pandas 사용 as로 별칭 붙여주기
import pandas as pd

# pandas 이용하여 파일 불러오기. 데이터 보기 편하게 head() 함수 이용
df1 = pd.read_csv('pop_kor.csv')
df2 = pd.read_excel('관서별 5대범죄 발생 및 검거.xlsx')

df1.head()
df2.head()

# 관서명을 구 이름으로 매핑하기 위해 매핑할 데이터 입력 후 map()사용하여 매핑. 결측지는 '구 없음'으로 표출
map_data = {
    '서대문서': '서대문구', '수서서': '강남구', '강서서': '강서구', '서초서': '서초구',
    '서부서': '은평구', '중부서': '중구', '종로서': '종로구', '남대문서': '중구',
    '혜화서': '종로구', '용산서': '용산구', '성북서': '성북구', '동대문서': '동대문구',
    '마포서': '마포구', '영등포서': '영등포구', '성동서': '성동구', '동작서': '동작구',
    '광진서': '광진구', '강북서': '강북구', '금천서': '금천구', '중랑서': '중랑구',
    '강남서': '강남구', '관악서': '관악구', '강동서': '강동구', '종암서': '성북구',
    '구로서': '구로구', '양천서': '양천구', '송파서': '송파구', '노원서': '노원구',
    '방배서': '서초구', '은평서': '은평구', '도봉서': '도봉구'
}

df2['구별'] = df2['관서명'].map(map_data).fillna('구 없음')

# index 사용하여 구별 데이터로 변경하고 같은 구인 경우 aggfunc='sum' 이용하여 더해주기
# 기존 데이터 '관서명'으로 인덱스 되어있던거 아니라서 필요없는 컬럼인 관서명은 drop()이용하여 열 삭제(axis=1)
# pivot = df2.drop(columns=['관서명']).pivot_table(index='구별', aggfunc='sum')
pivot = df2.drop('관서명', axis=1).pivot_table(index='구별', aggfunc='sum')
pivot

# drop() 이용 하여 '구 없음'에 해당하는 행 삭제하기
# 인덱스 삭제할 때, 삭제하려는 항목이 존재하지 않아도 오류를 발생시키지 않도록 errors='ignore' 사용하기
pivot = pivot.drop(index='구 없음', errors='ignore')

# 각 범죄 별로 검거율 계산하고, 각 검거율 데이터 컬럼 추가하기
# for문 사용해서 crime 변수에 공통 단어 넣어주기
for crime in ['강간', '강도', '살인', '절도', '폭력']:
    # f-string을 이용해 컬럼명 생성 
    # 각 범죄 유형에 대해 검거율(= 검거 건수 / 발생 건수 * 100)을 계산해서 컬럼 생성
    pivot[f'{crime}검거율'] = (pivot[f'{crime}(검거)'] / pivot[f'{crime}(발생)']) * 100

# 소계에 대한 검거율 컬럼 추가
pivot['검거율'] = (pivot['소계(검거)'] / pivot['소계(발생)']) * 100

# del pivot['강간(검거)'] : del의 경우 단일 열만 지정가능하여 필요없는 컬럼 drop() 이용해서 삭제
del_column = ['강간(검거)', '강도(검거)', '살인(검거)', '절도(검거)', '폭력(검거)', '소계(발생)', '소계(검거)']
# inplace=True 사용하여 원본 테이블에 적용
pivot.drop(del_column, axis=1, inplace=True)
# pivot.drop(columns=del_column, inplace=True)

# rename()이용해서 컬럼명 변경
rename_column = {
    '강간(발생)': '강간', '강도(발생)': '강도', '살인(발생)': '살인',
    '절도(발생)': '절도', '폭력(발생)': '폭력'
}
# pivot.rename(columns=rename_column, inplace=True)
pivot.rename(rename_column, axis=1, inplace=True)

# 인구 데이터 불러오고 출력하기 head()이용해서 위에서부터 5개까지만
# index_col='구별' 인덱스 셋팅해서 불러오기
pop_data = pd.read_csv('pop_kor.csv', index_col='구별')
pop_data.head()

# join() 사용해서 피벗 테이블과 merge
merge_table = pivot.join(pop_data, how='inner')

# 검거율 기준으로 오름차순 정렬하기
sort_table = merge_table.sort_values(by='검거율')
sort_table

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율,인구수
구별,,,,,,,,,,,,
관악구,320,12,9,2706,3298,69.062500,116.666667,88.888889,30.561715,80.109157,58.502758,529031
동작구,285,9,5,1865,1910,48.771930,55.555556,100.000000,35.442359,83.089005,58.836524,412774
중구,170,9,3,2548,2224,65.294118,66.666667,66.666667,33.712716,88.309353,59.386354,134329
강동구,156,6,4,2366,2712,78.846154,133.333333,75.000000,33.347422,82.890855,60.469108,463321
영등포구,295,22,14,2964,3572,62.033898,90.909091,85.714286,32.995951,82.894737,60.492209,417811
송파구,220,13,11,3239,3295,80.909091,76.923077,90.909091,34.856437,84.552352,60.681617,667480
마포구,294,14,8,2555,2983,84.013605,71.428571,100.000000,31.819961,84.445189,61.445166,398351
구로구,281,15,8,2335,3007,58.362989,73.333333,75.000000,38.072805,80.877951,62.026213,454604
양천구,120,6,3,1890,2509,87.500000,50.000000,166.666667,35.555556,80.908729,62.168728,489010
